In [1]:
links = [
    'https://chinapost.nownews.com/20210716-2654837',
    'https://chinapost.nownews.com/20210716-2654495',
    'https://chinapost.nownews.com/20210712-2648271',
    'https://chinapost.nownews.com/20210719-2658483',
    'https://chinapost.nownews.com/20210719-2654835'
]

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import fasttext

In [3]:
path_to_pretrained_model = './tmp/lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)

In [4]:
def scrap_and_clean_text(link):
    source = urlopen(link).read()
    soup = BeautifulSoup(source,'lxml')
    text = ''
    for paragraph in soup.find_all('p'):
        text += paragraph.text
        
    text = re.sub(r'【.*?】+', '', text)
    text = re.sub(r"^\s+", "", text)
    
    rx = r'(?:(?<=[.!?。」])|(?<=[.!?]["”」]))'
    list_text = []

    for result in re.split(rx, text):
        list_text.append(result)
    
    for i in range(len(list_text)):
      if len(list_text[i]) <=3:
        list_text[i-1] += list_text[i]

    list_text = [x for x in list_text if not len(x) <= 3]
    
    for _ in range(5):
        for i in range(1,len(list_text)):
          if list_text[i][0] == ' ':
            list_text[i-1] += list_text[i]

        list_text = [x for x in list_text if not x[0] == ' ']
    
    for _ in range(5): 
        for i in range(len(list_text)-1):
          lang1 = fmodel.predict(list_text[i])[0][0]
          lang2 = fmodel.predict(list_text[i+1])[0][0]
            
          if lang1 == '__label__ja':
            lang1 = '__label__zh'
        
          if lang1 == lang2:
            list_text[i] += list_text[i+1]
            list_text[i+1] = ''
        list_text = [x for x in list_text if not x == '']
    
    
    en = []
    zh = []
    i=0

    while i <= len(list_text)-1:
        if i % 2 == 0:
            zh.append(list_text[i])
        else:
            en.append(list_text[i])
        i += 1
    
    return en, zh

In [5]:
en_all = []
zh_all = []

for link in links:
    en = []
    zh = []
    en, zh = scrap_and_clean_text(link)
    en_all.extend(en)
    zh_all.extend(zh)

In [6]:
len(en_all)

43

In [7]:
len(zh_all)

43

In [8]:
en_all

['Frontline medical workers have experienced long, stressful working hours since Taiwan entered the Level 3 epidemic warning.Against this backdrop, many people have contacted local medical centers and clinics to express their gratitude, including Taiwan-based Russian YouTuber “Broccoli Vlog” (花椰菜).',
 'The popular YouTuber on Friday shared a video of himself and his friends, John and Tony, preparing 300 cinnamon rolls for the staff of Linkou Chang Gung Memorial Hospital (林口長庚紀念醫院).',
 'Speaking about the two-day preparation, John explained that he stayed in mainland China for 13 years before coming to Taiwan. Upon arrival here, he discovered numerous cultural differences and kept track of them in a notebook.',
 'He remarked that some Chinese characters “look the same” but they have a “different pronunciation.” After finding his marks here, he said with a smile that he has had a better understanding of Taiwan.',
 'One of his friends, who helped contact the hospital, stressed that Taiwan

In [9]:
zh_all

['隨著三級警戒不斷延長，前線醫護人員的壓力也不斷增加，也因此許多人為了感謝辛苦的醫療人員以各種方式表達小心意替他們加油打氣。',
 '來自俄羅斯的台灣YouTuber 「花椰菜」於上週五 (9日)，與他的朋友們Jony和Tony決定手做300個肉桂捲送給林口長庚紀念醫院。',
 '在製作的過程中，John透露他之前在中國住了13年，剛來台灣的時候還有許多不適應的地方，為此他還準備了一本筆記專門紀錄兩地的文化差異。',
 '他解釋道，很多字在兩地寫起來一樣，但是唸法卻不盡相同，久而久之，現在他已經相當適應台灣的生活。',
 '另外一位好友則說道，台灣的醫護人員正在經歷一段辛苦的時期，如果能夠透過自己的行動帶給他們鼓勵的話，那樣非常值得。',
 '影片的結尾，他們將300個肉桂捲裝箱送到醫院門口。雖然在疫情期間外國人無法獲得補助，但是他們依舊願意為台灣的疫情盡一份力，而這善良的舉動也感動了無數網友。',
 '如果我的免疫力低，疫苗對我會有效嗎？對於較缺乏抵抗力的族群，疫苗效果也許沒有一般正常人來得好，但是仍會提供一定程度的保護力。',
 '這也是為什麼免疫系統因疾病或因服用特殊藥物下降的人仍應接種疫苗，而周遭的親朋好友是否接種疫苗，也會成為能否成功阻止病毒擴散的重要關鍵。',
 '在美國，大約3%的成人免疫力低弱，其中包括愛滋病(HIV or AIDS)病患、器官移植者、癌症病患，以及患有類風濕性關節炎 (rheumatoid arthritis)、發炎性腸道疾病 (inflammatory bowel disease)、紅斑性狼瘡 (lupus)等自體免疫疾等病患。',
 'COVID-19疫苗雖沒有針對大批免疫系統衰弱的族群做過研究，然而從過去流感疫苗與肺炎疫苗有限的資料和經驗可判斷，疫苗接種效果可能不如身體較為健康的一般大眾。這也意味著免疫力不足的民眾外出時仍應戴口罩並避免人群接觸，做好防護措施。',
 '位於西雅圖華盛頓大學醫學院的器官移植專家Ajit Limaye醫師指出：「接種疫苗後仍需保持未接種前的防疫習慣 (例如保持社交距離和戴口罩)，才能真正降低感染風險。」',
 '雖然大部分的癌症患者都應盡快接種疫苗，但是根據美國國家癌症資訊網所述，接受骨髓移植 (stem cell transplant) 或CAR-T細胞治療 (CAR T-cell thera

In [10]:
with open('english_ds.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in en_all:
        filehandle.write('%s\n' % listitem)

In [11]:
with open('chinese_ds.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in zh_all:
        filehandle.write('%s\n' % listitem)